In [284]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [285]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    #T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    #WS160: float      # Wind speed at 160 m [m/s]
    #WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    #WL_tot: float     # Total water level [m MSL]
    #Hmax: float       # Maximum wave height [m]
    #T_Hmax: float     # Wave period associated with Hmax [s]
    #Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    #Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    #WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

_ = MetOceanParameters(
    Hm0=5.9,
    Tp=15.3, #nog aanpassen! HOGERE Tp ZORGT VOOR GROTERE SCOUR DEPTH
    #T02=9.8,
    CS_total=1.2,
    #WS160=29.7,
    #WS160_adj=31.8,
    #WL_tot=1.3,
    #Hmax=19.2,
    #T_Hmax=12.1,
    #Cmax_SWL=12.8,
    #Cmax_MSL=14.6,
    WL_total=2.4, #nog aanpassen! AFHANKELIJK VAN SITUATIE
    #WL_residual=0
)

## Combined current & waves

In [286]:
def soulsby_um(H_m0, T_p, h, g=9.81):
    """
    Compute U_m,Soulsby using the Soulsby formula.

    Parameters
    ----------
    H_m0 : float or ndarray
        Significant wave height (m)
    T_p : float or ndarray
        Peak wave period (s)
    h : float or ndarray
        Water depth (m)
    g : float, optional
        Gravitational acceleration (m/s^2), default 9.81

    Returns
    -------
    U_m : float or ndarray
        Maximum orbital velocity (m/s)
    """
    term1 = H_m0 / (2 * np.sqrt(2))
    term2 = np.sqrt(g / h)
    exponent = -((4.75 / T_p) * np.sqrt(h / g))**2.1

    U_m = term1 * term2 * np.exp(exponent)
    return U_m

In [287]:
def relative_velocity(u_c, U_w_bed):
    """
    Compute relative velocity and interpret flow regime (scalar version).
    """
    U_rel = u_c / (u_c + U_w_bed)
    return U_rel

In [288]:
bed_um = soulsby_um(H_m0=_.Hm0, T_p=_.Tp, h=40 + _.WL_total)
print(f"Bed orbital velocity (U_m): {bed_um:.2f} m/s")
rel_u = relative_velocity(_.CS_total, bed_um)
print(f"Relative velocity: {rel_u:.2f}")

Bed orbital velocity (U_m): 0.67 m/s
Relative velocity: 0.64


## Scour depth

In [289]:
def initial_scour_depth(D, h_0):
    h_s = 2 * D * np.tanh(h_0 / D)
    return h_s

## Scour depth Waves & Current

In [290]:
g = 9.81
D = 9.0 # Pile diameter [m]

def wave_number(T, h, tol=1e-6, max_iter=100):
    """
    Solve dispersion relation: omega^2 = g k tanh(kh)
    """
    omega = 2 * np.pi / T
    k = omega**2 / g  # deep-water initial guess

    for _ in range(max_iter):
        f = g * k * np.tanh(k * h) - omega**2
        df = g * np.tanh(k * h) + g * k * h / np.cosh(k * h)**2
        k_new = k - f / df
        if abs(k_new - k) < tol:
            return k_new
        k = k_new

    raise RuntimeError("Dispersion relation did not converge")


def orbital_velocity_bed(H, T, h):
    """
    Maximum orbital velocity at the seabed
    """
    k = wave_number(T, h)
    u_hat_b = (np.pi * H) / (T * np.sinh(k * h))
    return u_hat_b, k


def equilibrium_scour_depth(
    D,
    h,
    H,
    T,
    u_c,
    h_p,
):
    """
    Compute equilibrium scour depth Seq [m]
    """

    # Orbital velocity & KC
    u_hat_b, k = orbital_velocity_bed(H, T, h)
    KC = u_hat_b * T / D

    # Relative velocity
    U_rel = u_c / (u_c + u_hat_b)

    # Wave reduction factor
    A = 0.012 * KC + 0.57 * KC**1.77 * U_rel**3.76
    K_w = 1 - np.exp(-A)

    # Pile height reduction factor
    K_h = (h_p / h) ** 0.67

    # Equilibrium scour depth
    S_eq = 1.5 * D * np.tanh(h / D) * K_w * K_h

    return {
        "Seq": S_eq,
        "KC": KC,
        "u_hat_b": u_hat_b,
        "U_rel": U_rel,
        "K_w": K_w,
        "K_h": K_h,
        "k": k,
    }

In [291]:
result = equilibrium_scour_depth(
    D=D,      # paaldiameter [m]
    h=40 + _.WL_total,     # waterdiepte [m]
    H=_.Hm0,      # Hm0 [m]
    T=_.Tp,     # Tp [s]
    u_c=_.CS_total,    # dieptegemiddelde stroming [m/s]
    h_p=40 + _.WL_total,   # onderwater paalhoogte [m]
)

print("Equilibrium scour depth:")
print(f"Seq = {result['Seq']:.2f} m")
print(f"KC  = {result['KC']:.2f}")
print(f"û_b = {result['u_hat_b']:.2f} m/s")
print(f"U_rel = {result['U_rel']:.2f}")
print(f"K_w = {result['K_w']:.4f}")
print(f"K_h = {result['K_h']:.4f}")
print(f"k = {result['k']:.4f} 1/m")

Equilibrium scour depth:
Seq = 2.13 m
KC  = 1.82
û_b = 1.07 m/s
U_rel = 0.53
K_w = 0.1575
K_h = 1.0000
k = 0.0229 1/m


In [292]:
#Eckert's formula - Extra check
T_p = _.Tp
depth = 40 + _.WL_total

omega = 2 * np.pi / T_p
alpha = (omega**2 * depth) / g
k = alpha / (np.tanh(alpha))**0.5 * 1/depth

print(f"k = {k:.4f} 1/m")

k = 0.0218 1/m


In [293]:
result_HWL = equilibrium_scour_depth(
    D=9.0,      # paaldiameter [m]
    h=(40+2.4), # waterdiepte [m]
    H=5.9,      # Hm0 [m]
    T=15.3,     # Tp [s]
    u_c=1.2,    # dieptegemiddelde stroming [m/s]
    h_p=42.4,   # onderwater paalhoogte [m]
)

print("Equilibrium scour depth HWL:")
print(f"Seq = {result_HWL['Seq']:.2f} m")
print(f"KC  = {result_HWL['KC']:.2f}")
print(f"û_b = {result_HWL['u_hat_b']:.2f} m/s")
print(f"U_rel = {result_HWL['U_rel']:.2f}")
print(f"K_w = {result_HWL['K_w']:.4f}")
print(f"K_h = {result_HWL['K_h']:.4f}")
print(f"k = {result_HWL['k']:.4f} 1/m")

print()

result_LWL = equilibrium_scour_depth(
    D=9.0,      # paaldiameter [m]
    h=(40-1.9), # waterdiepte [m]
    H=5.9,      # Hm0 [m]
    T=15.3,     # Tp [s]
    u_c=1.2,    # dieptegemiddelde stroming [m/s]
    h_p=(40-1.9),   # onderwater paalhoogte [m]
)

print("Equilibrium scour depth LWL:")
print(f"Seq = {result_LWL['Seq']:.2f} m")
print(f"KC  = {result_LWL['KC']:.2f}")
print(f"û_b = {result_LWL['u_hat_b']:.2f} m/s")
print(f"U_rel = {result_LWL['U_rel']:.2f}")
print(f"K_w = {result_LWL['K_w']:.4f}")
print(f"K_h = {result_LWL['K_h']:.4f}")
print(f"k = {result_LWL['k']:.4f} 1/m")

Equilibrium scour depth HWL:
Seq = 2.13 m
KC  = 1.82
û_b = 1.07 m/s
U_rel = 0.53
K_w = 0.1575
K_h = 1.0000
k = 0.0229 1/m

Equilibrium scour depth LWL:
Seq = 2.14 m
KC  = 1.98
û_b = 1.17 m/s
U_rel = 0.51
K_w = 0.1587
K_h = 1.0000
k = 0.0239 1/m
